<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/Assignment-1/Question%2010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
from keras.datasets import mnist
import numpy as np
from  matplotlib import pyplot as plt
import time
import math
import wandb
wandb.login()


In [ ]:
((xtrain, ytrain), (xtest, ytest)) = mnist.load_data()

items=["0", "1","2", "3", "4", "5", "6", "7", "8","9"]
ftrx=np.zeros((60000,784))
ftry=np.zeros((60000,10))
ftex=np.zeros((10000,784))
ftey=np.zeros((10000,10))
for i in range(len(xtrain)):
  ftrx[i]=xtrain[i].flatten();
  ftrx[i]=ftrx[i]/256
  ftry[i][ytrain[i]]=1
for i in range(len(xtest)):
  ftex[i]=xtest[i].flatten();
  ftex[i]=ftex[i]/256
  ftey[i][ytest[i]]=1





In [ ]:

     # activation functions and gradients

def sigmoid(x):
  return np.where(x>=0, 1/(1+np.exp(-x)), np.exp(x)/(1+np.exp(x)))

def grad_sigmoid(z):
  return z*(1-z)

def softmax(z):
  mx=np.max(z)
  z=z-mx
  out=(np.exp(z)+1e-9)/np.sum(np.exp(z)+1e-9)
  return out

def grad_softmax(yh,ty):
  return yh-ty

def Relu(x):
    return np.maximum(0, x)

def grad_Relu(x):
    return [1 if xi>0 else 0 for xi in x]

def Tanh(x):
  a= 2*np.where(x>=0, 1/(1+np.exp(-2*x)), np.exp(2*x)/(1+np.exp(2*x)))
  return a-1


def grad_Tanh(x):
    return 1-Tanh(x)**2

def InitParams(x,init):
    weights=[]
    baises=[]
    if(init=='random'):
      weights.append((np.random.randn(x[0],784)))
      baises.append((np.random.randn(x[0])))
      l=len(x)-1
      for i in range(l):
          weights.append(0.1*(np.random.randn(x[i+1],x[i])))
          baises.append(0.1*(np.random.randn(x[i+1])))
      weights.append(0.1*(np.random.randn(10,x[-1])))
      baises.append(0.1*(np.random.randn(10)))
    else:
      weights.append(np.random.randn(x[0], 784)*np.sqrt(2./(784)))
      baises.append(np.zeros(x[0]))
      l=len(x)-1
      for i in range(l):
          weights.append(np.random.randn(x[i+1], x[i])*np.sqrt(2./(x[i])))
          baises.append(np.zeros(x[i+1]))
      weights.append(np.sqrt(2./(x[-1]))*(np.random.randn(10,x[-1])))
      baises.append((np.zeros(10)))


    return weights,baises



def initgrads(N,Nunits):
  dh,da,dw,db=[],[],[],[]
  dw.append(np.zeros((Nunits[0],784)))
  db.append(np.zeros(Nunits[0]))
  da.append(np.zeros(Nunits[0]))
  dh.append(np.zeros(Nunits[0]))
  for i in range(1,N):
    dw.append(np.zeros((Nunits[i],Nunits[i-1])))
    db.append(np.zeros(Nunits[i]))
    da.append(np.zeros(Nunits[i]))
    dh.append(np.zeros(Nunits[i]))
  dw.append(np.zeros((10,Nunits[-1])))
  db.append(np.zeros(10))
  da.append(np.zeros(10))
  dh.append(np.zeros(10))
  return dw,db,da,dh

  
def Feed_Frwd_Nw1(xTr,Weighs,Bais,activation,loss):
  a_i=[]
  h_i=[]
  a_i.append(np.dot(Weighs[0],xTr)+Bais[0])       
  if (activation=='sig'):
    h_i.append(sigmoid(a_i[0]))
  elif (activation=='tanh'):
    h_i.append(Tanh(a_i[0]))
  elif (activation=='relu'):
    h_i.append(Relu(a_i[0]))
  for i in range(1,len(Weighs)-1):
    a_i.append((np.dot(Weighs[i],h_i[i-1])+Bais[i]))
    if (activation=='sig'):
      h_i.append(sigmoid(a_i[i]))
    elif (activation=='tanh'):
      h_i.append(Tanh(a_i[i]))
    elif (activation=='relu'):
      h_i.append(Relu(a_i[i]))
  a_i.append(np.dot(Weighs[-1],h_i[-1])+Bais[-1])
  h_i.append(softmax(a_i[-1])) 
  yp=h_i[-1]
  return yp,a_i,h_i


def Back_Prop(ip,ypr,ty,a_i,h_i,W,B,N,nrl,activation,Loss):
  k=len(ypr)
  dw,db,da,dh=initgrads(N,nrl)
  if (Loss=="ce"):
    dh[-1]=[-(t/ypr) if t==1 else 0 for t in ty ]
    da[-1]=ypr-ty
  elif (Loss=="mse"):
    dh[-1]=(ypr-ty)
    da[-1]=dh[-1]*(ypr-ypr**2)

  db[-1]=da[-1] 
  dw[-1]=np.dot((da[-1][:,np.newaxis]),(h_i[-2][:,np.newaxis]).T)
 
  for i in range(N-1,-1,-1):
    dh[i]=np.squeeze(np.dot(W[i+1].T,da[i+1]))

    if (activation=='sig'):
      da[i]=dh[i]*grad_sigmoid(h_i[i])
    elif (activation=='tanh'):
      da[i]=dh[i]*grad_Tanh(a_i[i])
    elif (activation=='relu'):
      da[i]=dh[i]*grad_Relu(a_i[i])
    db[i]=np.copy(da[i])
    if (i==0):
      dw[i]=np.dot(da[i][:,np.newaxis],ip[:,np.newaxis].T)
    else:
      dw[i]=np.dot(da[i][:,np.newaxis],h_i[i-1][:,np.newaxis].T)
  return dw,db


def sum_weights(W):
  sum=0
  for i in range(len(W)):
    sum+=np.sum(W[i])
  return sum



           # Losses and accuracies
def val_acc_loss(W,B,activation,ls_fun,wd):
  cost=0
  count=0
  if (ls_fun=='ce'):
    for i in range(len(ftrx[54000:])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost += (np.sum(np.multiply(ftry[i],np.log(ypr))) + wd/2*sum_weights(W) )
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return -cost/6000, count/6000
    #return np.sum(ay.multiply(np.log(yh)))
  elif (ls_fun=='mse'):
    for i in range(len(ftrx[54000:])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost+=(0.5*np.sum((ftry[i]- ypr)**2) + wd/2*sum_weights(W) )
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return cost/6000, count/6000

def train_acc_loss(W,B,activation,ls_fun,wd):
  cost=0
  count=0
  if (ls_fun=='ce'):
    for i in range(len(ftrx[:54000])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost += (np.sum(np.multiply(ftry[i],np.log(ypr))) + wd/2*sum_weights(W) )
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return -cost/54000, count/54000
    #return np.sum(ay.multiply(np.log(yh)))
  elif (ls_fun=='mse'):
    for i in range(len(ftrx[:54000])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost+=(0.5*np.sum((ftry[i]- ypr)**2) + wd/2*sum_weights(W) )
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return cost/54000, count/54000




def test_acc(W,B,activation,ls_fun):
  count=0
  predictions=[]
  matrix=np.zeros((10,10))
  for i in range(len(ftex)):
    ypr,_,_=Feed_Frwd_Nw1(ftex[i],W,B,activation,ls_fun)
    if (np.argmax(ypr) == ytest[i]):
      count+=1
    predictions.append(np.argmax(ypr))
    matrix[ytest[i]][np.argmax(ypr)]+=1
    
  lables={}
  for i in range(len(items)):
    lables[i]=items[i]
  wandb.log({"confusion matrix": wandb.plot.confusion_matrix(probs=None,
                                                              y_true=ytest,
                                                              preds=predictions,
                                                              class_names=items)})
  wandb.log({"confusion matrix v2.0": wandb.plots.HeatMap(items, items, matrix, show_text=True),"Accuracy":count/len(ytest)})
  return count/len(ytest)



def gradDecent(ftrx,ftry,init,wd,eta,N,nrl,activation,Ls_fun,epochs,batchsize):
  W,B=InitParams(nrl,init)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          W[k]=np.subtract(W[k],eta*uw[k]/batchsize)
          B[k]=np.subtract(B[k],eta*ub[k]/batchsize)
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B


import copy 
def Momentum(ftrx,ftry,init,batchsize,gamma,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  pw,pb,_,_=initgrads(N,nrl)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          # z1,z2=W[k],B[k]
          vw[k]=np.add(gamma*pw[k],eta*uw[k]/batchsize)
          vb[k]=np.add(gamma*pb[k],eta*ub[k]/batchsize)
          W[k]=np.subtract(W[k],vw[k])
          B[k]=np.subtract(B[k],vb[k])
          pw[k]=copy.deepcopy(vw[k])
          pb[k]=copy.deepcopy(vb[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B

import copy 
def Nesterov(ftrx,ftry,init,batchsize,gamma,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  pw,pb,_,_=initgrads(N,nrl)
  w,b=copy.deepcopy(W),copy.deepcopy(B)
  e=0
  while(epochs>0):
    start = time.time()
    e+=1
    start = time.time()
    epochs-=1 
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx)):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun) 
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,w,b,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          # z1,z2=W[k],B[k]
          vw[k]=np.add(gamma*pw[k],eta*uw[k]/batchsize)
          vb[k]=np.add(gamma*pb[k],eta*ub[k]/batchsize)
          W[k]=np.subtract(W[k],vw[k])
          B[k]=np.subtract(B[k],vb[k])
          pw[k]=copy.deepcopy(vw[k])
          pb[k]=copy.deepcopy(vb[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k])
          for k in range(len(dw)):
            w[k]=np.subtract(W[k],gamma*pw[k])
            b[k]=np.subtract(B[k],gamma*pb[k])    
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B

def rmsprop(ftrx,ftry,init,batchsize,eps,beta,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          vw[k]=np.add(beta*vw[k],(1-beta)*uw[k]**2)
          vb[k]=np.add(beta*vb[k],(1-beta)*ub[k]**2)
          W[k]=np.subtract(W[k],((eta/batchsize)/np.sqrt(vw[k]+eps))*uw[k])
          B[k]=np.subtract(B[k],((eta/batchsize)/np.sqrt(vb[k]+eps))*ub[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B




def Adam(ftrx,ftry,init,batchsize,eps,beta1,beta2,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  mw,mb,_,_=initgrads(N,nrl)
  beta1=0.9
  e=0
  t=1
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        t+=1
        for k in range(len(gw)):
          vw[k]=np.add(beta2*vw[k],(1-beta2)*uw[k]**2)
          vb[k]=np.add(beta2*vb[k],(1-beta2)*ub[k]**2)

          mw[k]=np.add(beta1*mw[k],(1-beta1)*uw[k])
          mb[k]=np.add(beta1*mb[k],(1-beta1)*ub[k])
          mw[k]/=(1-np.power(beta1,t,dtype=np.float64))
          mb[k]/=(1-np.power(beta1,t,dtype=np.float64))
          vw[k]=np.divide(vw[k],(1-np.power(beta2,t,dtype=np.float64)))
          vb[k]=np.divide(vb[k],(1-np.power(beta2,t,dtype=np.float64)))
          W[k]=np.subtract(W[k],((eta/batchsize)/np.sqrt(vw[k]+eps))*mw[k])
          B[k]=np.subtract(B[k],((eta/batchsize)/np.sqrt(vb[k]+eps))*mb[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B








def NAdam(ftrx,ftry,init,batchsize,eps,beta1,beta2,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  mw,mb,_,_=initgrads(N,nrl)
  beta1=0.9
  e=0
  t=1
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        t+=1
        for k in range(len(gw)):
          vw[k]=np.add(beta2*vw[k],(1-beta2)*uw[k]**2)
          vb[k]=np.add(beta2*vb[k],(1-beta2)*ub[k]**2)

          mw[k]=np.add(beta1*mw[k],(1-beta1)*uw[k])
          mb[k]=np.add(beta1*mb[k],(1-beta1)*ub[k])
          mw[k]/=(1-np.power(beta1,t,dtype=np.float64))
          mb[k]/=(1-np.power(beta1,t,dtype=np.float64))
          vw[k]=np.divide(vw[k],(1-np.power(beta2,t,dtype=np.float64)))
          vb[k]=np.divide(vb[k],(1-np.power(beta2,t,dtype=np.float64)))
          W[k]=np.subtract(W[k],((eta/batchsize)/np.sqrt(vw[k]+eps))*((beta1*mw[k]+(1-beta1)*uw[k])/(1-beta1**t)))
          B[k]=np.subtract(B[k],((eta/batchsize)/np.sqrt(vb[k]+eps))*((beta1*mb[k]+(1-beta1)*ub[k])/(1-beta1**t)))
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B


In [ ]:
wandb.init(project="mnist")

In [ ]:
epochs = 15                                       
batchsize = 32                                                    
weightDecay = 0.00005                                           
hidden_layer_size = 128
weights_initializer='Xaviuor'                               
lr=0.0095
activation='tanh'                                            
optimizer = "adam"                      
num_hidden_layers= 3           
lossfun = "mse"

wandb.run.name="e_{}_hls_{}_numhl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}_learning_rate_{}_wdecay_{}".format(epochs,\
                                                                      hidden_layer_size,\
                                                                      num_hidden_layers,\
                                                                      optimizer,\
                                                                      batchsize,\
                                                                      weights_initializer,\
                                                                     activation,\
                                                                      lossfun,\
                                                                      lr,\
                                                                      weightDecay)
nrl=[hidden_layer_size for i in range(num_hidden_layers)]
param=Adam(ftrx,ftry,weights_initializer,batchsize,1e-8,0.9,0.9,weightDecay,lr,num_hidden_layers,nrl,activation,lossfun,epochs)
  

wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Visualizing heatmap.


In [ ]:
wandb.init(project="mnist")

Accuracy,▁
Train_acc,▁▄▆▆▇▇▇▇███████
Train_loss,█▅▃▃▂▂▂▂▁▁▁▁▁▁▁
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
val_Accuracy,▁▄▆▆▇▇▇▇███████
val_loss,█▅▃▃▂▂▂▂▁▁▁▁▁▁▁
Accuracy,0.9709
Train_acc,0.9857
Train_loss,0.01138
epoch,15
val_Accuracy,0.9855


In [ ]:
epochs = 15                                       
batchsize = 32                                                    
weightDecay = 0.00005                                           
hidden_layer_size = 64
weights_initializer='Xaviuor'                               
lr=0.0055
activation='tanh'                                            
optimizer = "nadam"                      
num_hidden_layers= 1           
lossfun = "ce"

wandb.run.name="e_{}_hls_{}_numhl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}_learning_rate_{}_wdecay_{}".format(epochs,\
                                                                      hidden_layer_size,\
                                                                      num_hidden_layers,\
                                                                      optimizer,\
                                                                      batchsize,\
                                                                      weights_initializer,\
                                                                     activation,\
                                                                      lossfun,\
                                                                      lr,\
                                                                      weightDecay)
nrl=[hidden_layer_size for i in range(num_hidden_layers)]
param=NAdam(ftrx,ftry,weights_initializer,batchsize,1e-8,0.9,0.9,weightDecay,lr,num_hidden_layers,nrl,activation,lossfun,epochs)

wandb: Visualizing heatmap.


In [11]:
wandb.init(project="mnist")

Accuracy,▁
Train_acc,▁▄▅▆▆▆▇▇▇▇█████
Train_loss,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
val_Accuracy,▁▄▅▆▆▆▇▇▇▇█████
val_loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁
Accuracy,0.9624
Train_acc,0.96846
Train_loss,0.1115
epoch,15
val_Accuracy,0.97333


In [12]:
epochs = 15                                       
batchsize = 32                                                    
weightDecay = 0.00005                                           
hidden_layer_size = 32
weights_initializer='Xaviuor'                               
lr=0.04
activation='relu'                                            
optimizer = "nadam"                      
num_hidden_layers= 2           
lossfun = "ce"

wandb.run.name="e_{}_hls_{}_numhl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}_learning_rate_{}_wdecay_{}".format(epochs,\
                                                                      hidden_layer_size,\
                                                                      num_hidden_layers,\
                                                                      optimizer,\
                                                                      batchsize,\
                                                                      weights_initializer,\
                                                                     activation,\
                                                                      lossfun,\
                                                                      lr,\
                                                                      weightDecay)
nrl=[hidden_layer_size for i in range(num_hidden_layers)]
param=NAdam(ftrx,ftry,weights_initializer,batchsize,1e-8,0.9,0.9,weightDecay,lr,num_hidden_layers,nrl,activation,lossfun,epochs)

wandb: Visualizing heatmap.
